# Real Time Algorithm

## Load Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time

## Import Data

In [2]:
df = pd.read_csv("/Users/constancabraga/Desktop/IE University/TERM 3/Captsone Project/Gestamp/line1.csv")

In [3]:
df

,TimeStamp,SignalName,Value
0,2022-01-01 00:00:08.190773,oil_level_1,5716.2188
1,2022-01-01 00:00:11.004477,oil_level_1,5715.0
2,2022-01-01 00:00:12.386434,oil_level_1,5717.0312
3,2022-01-01 00:00:13.761420,oil_level_1,5716.2188
4,2022-01-01 00:00:15.518799,oil_level_1,5717.0312
...,...,...,...
10683664,2022-03-31 14:31:31.013587,oil_level_1,5709.3125
10683665,2022-03-31 14:31:31.393079,oil_level_1,5712.9688
10683666,2022-03-31 14:31:31.774115,oil_level_1,5762.9375
10683667,2022-03-31 14:31:32.145733,oil_level_1,5745.0625


## Real Time Algorithm

In [4]:
counter = 0
WARMUP = 50
HISTORICAL = 100
last_ref = 'nan'
last_mainpumps = 'nan'
last_numstroke = 'nan'
oil_array = []
mean_array = []
last_mean = 'nan'
oil_leak = []
leak_alarm = []

for i in df.index:
    #print(i)
    aux_df = df.loc[[i]]
    if aux_df['SignalName'][i] == 'ref_1':
        last_ref = aux_df['Value'].values[0]
        print('Ref changed', i,' to value ', last_ref)
        mean_array = []
        
    elif aux_df['SignalName'][i] == 'mainpumps_ok_1':
        last_mainpumps = aux_df['Value'].values[0]
        #print('mainpumps changed', i, ' to value ', last_mainpumps)
        if last_mainpumps == 'false':
            mean_array = []
            
    elif aux_df['SignalName'][i] == 'oil_level_1':
        if last_ref != 'nan' and last_mainpumps == 'true' and last_numstroke != 'nan': #Initialize only when all variables are differente from NAN
           # print('Oil calculation')
            oil_array.append(float(aux_df['Value'].values[0]))
            
    elif aux_df['SignalName'][i] == 'numstroke_1':
       # print('numstroke changed', i)
        last_numstroke = aux_df['Value'].values[0]
        if len(mean_array) <= WARMUP:
            mean_array.append(np.mean(oil_array))
            oil_array = []
        elif len(mean_array) > WARMUP: 
            last_mean = np.mean(oil_array)
            oil_array = []
            if WARMUP < len(mean_array) <= HISTORICAL:
                mean = np.mean(mean_array)
                oil_limit = mean - 50
                if oil_limit>=last_mean:
                    counter = counter + 1
                    print(f'OIL LEAK ALARM number {counter}')
                    oil_leak.append(1)
                else:  
                    oil_leak.append(0)
            elif len(mean_array) > HISTORICAL:
                mean_array.append(last_mean)
                mean_array = mean_array[1:]
        

/Users/constancabraga/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/constancabraga/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Ref changed 569053  to value  1
Ref changed 599134  to value  6
Ref changed 954097  to value  2
Ref changed 1260162  to value  1
Ref changed 1260815  to value  2
Ref changed 1260905  to value  1
Ref changed 1361218  to value  3
Ref changed 1421863  to value  2
Ref changed 1591055  to value  6
Ref changed 1889323  to value  5
OIL LEAK ALARM number 1
OIL LEAK ALARM number 2
Ref changed 2131737  to value  6
Ref changed 2558523  to value  5
Ref changed 2788161  to value  2
Ref changed 3157897  to value  6
Ref changed 3316870  to value  2
Ref changed 3755845  to value  5
Ref changed 3913720  to value  2
Ref changed 4044655  to value  6
Ref changed 4193369  to value  5
Ref changed 4243988  to value  2
Ref changed 4659623  to value  2
OIL LEAK ALARM number 3
Ref changed 4796481  to value  6
Ref changed 4948048  to value  5
Ref changed 5029652  to value  2
Ref changed 5453265  to value  5
Ref changed 5603142  to value  2
Ref changed 5770082  to value  1
Ref changed 5868512  to value  2
Ref cha